# XYI GST circuit generation and conversion

Leon Maurer

2020-05-11

In [1]:
import cirq
import pygsti
from pygsti.modelpacks import smq1Q_XYI
import numpy as np
import tqdm

## 1. Generate the GST circuits

### Make target gate set $\{\sqrt{X},\sqrt{Y},I\}$

In [2]:
target_model = smq1Q_XYI.target_model()

### Preparation and measurement fiducials, germs

In [3]:
preps = smq1Q_XYI.prep_fiducials()
effects = smq1Q_XYI.meas_fiducials()
germs = smq1Q_XYI.germs()

In [4]:
preps

[Circuit({}@(0)),
 Circuit(Gxpi2:0@(0)),
 Circuit(Gypi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0Gxpi2:0@(0)),
 Circuit(Gypi2:0Gypi2:0Gypi2:0@(0))]

In [5]:
effects

[Circuit({}@(0)),
 Circuit(Gxpi2:0@(0)),
 Circuit(Gypi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0Gxpi2:0@(0)),
 Circuit(Gypi2:0Gypi2:0Gypi2:0@(0))]

In [6]:
germs

[Circuit([]@(0)),
 Circuit(Gxpi2:0@(0)),
 Circuit(Gypi2:0@(0)),
 Circuit(Gxpi2:0Gypi2:0@(0)),
 Circuit(Gxpi2:0Gxpi2:0Gypi2:0@(0))]

### Construct pyGSTi circuits

This takes a few seconds.

In [7]:
max_lengths = list(np.logspace(0, 10, 11, base=2, dtype=int))

In [8]:
print(max_lengths)

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]


In [9]:
circuits = pygsti.construction.make_lsgst_experiment_list(target_model, preps, effects, germs, max_lengths)
dict(enumerate(circuits))

AttributeError: cannot import name 'make_lsgst_experiment_list' from 'pygsti.construction'

In [10]:
len(circuits)

1624

In [27]:
circuit = circuits[111]

In [29]:
print(circuit)

Qubit 0 ---|Gxpi2|-|Gxpi2|-| |-| |-|Gxpi2|---



In [33]:
circuit.get_layer_with_idles(1)

(Label[Gxpi2:0],)

In [34]:
circuit.get_layer_with_idles(2)

()

## 2. Convert to runable `cirq.Circuit`'s

### Setup

Now, we need to map the qubit names from pyGSTi (`0`, `1`, etc.) into cirq qubits. Here, there's only one qubit to convert. I'm using `cirq.GridQubit(8, 3)` because that was used in Kevin's notebook.

In [11]:
qubit_label_dict = {0: cirq.GridQubit(8, 3)}

### Testing examples

Do an example conversion.

In [12]:
pygsti_circuit = circuits[111]
print('pyGSTi:')
print(pygsti_circuit)
print('Cirq:')
print(pygsti_circuit.convert_to_cirq(qubit_label_dict))

pyGSTi:
Qubit 0 ---|Gxpi2|-|Gxpi2|-| |-| |-|Gxpi2|---

Cirq:
(8, 3): ───X^0.5───X^0.5───────────X^0.5───


Do another example conversion.

In [13]:
pygsti_circuit = circuits[90]
print('pyGSTi:')
print(pygsti_circuit)
print('Cirq:')
print(pygsti_circuit.convert_to_cirq(qubit_label_dict))

pyGSTi:
Qubit 0 ---|Gypi2|-|Gypi2|-|Gypi2|-|Gypi2|-|Gxpi2|-|Gxpi2|-|Gxpi2|---

Cirq:
(8, 3): ───Y^0.5───Y^0.5───Y^0.5───Y^0.5───X^0.5───X^0.5───X^0.5───


Now, lets try the same thing but specifing a wait duration for the idle operation.

In [14]:
wait_duration = cirq.Duration(nanos=100)

In [15]:
pygsti_circuit = circuits[111]
print('pyGSTi:')
print(pygsti_circuit)
print('Cirq:')
print(pygsti_circuit.convert_to_cirq(qubit_label_dict, wait_duration))

pyGSTi:
Qubit 0 ---|Gxpi2|-|Gxpi2|-| |-| |-|Gxpi2|---

Cirq:
(8, 3): ───X^0.5───X^0.5───────────X^0.5───


In [16]:
pygsti_circuit = circuits[90]
print('pyGSTi:')
print(pygsti_circuit)
print('Cirq:')
print(pygsti_circuit.convert_to_cirq(qubit_label_dict, wait_duration))

pyGSTi:
Qubit 0 ---|Gypi2|-|Gypi2|-|Gypi2|-|Gypi2|-|Gxpi2|-|Gxpi2|-|Gxpi2|---

Cirq:
(8, 3): ───Y^0.5───Y^0.5───Y^0.5───Y^0.5───X^0.5───X^0.5───X^0.5───


### The real thing

Now, do the real conversion.

In [17]:
cirq_circuits = [c.convert_to_cirq(qubit_label_dict, wait_duration) for c in tqdm.tqdm(circuits)]

100%|██████████| 1624/1624 [00:06<00:00, 236.63it/s]


Note that we're missing the measurments, the idle operations don't have a time associated with them, and the first circuit is empty (it's should just be an idle). Otherwise, the results look good, and those things should be easy to fix.

In [18]:
cirq_circuits

[,
 (8, 3): ───X^0.5───,
 (8, 3): ───Y^0.5───,
 (8, 3): ───X^0.5───X^0.5───,
 (8, 3): ───X^0.5───X^0.5───X^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───Y^0.5───,
 (8, 3): ───X^0.5───Y^0.5───,
 (8, 3): ───X^0.5───X^0.5───X^0.5───X^0.5───,
 (8, 3): ───X^0.5───Y^0.5───Y^0.5───Y^0.5───,
 (8, 3): ───Y^0.5───X^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───,
 (8, 3): ───Y^0.5───X^0.5───X^0.5───,
 (8, 3): ───Y^0.5───X^0.5───X^0.5───X^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───Y^0.5───Y^0.5───,
 (8, 3): ───X^0.5───X^0.5───Y^0.5───,
 (8, 3): ───X^0.5───X^0.5───X^0.5───X^0.5───X^0.5───,
 (8, 3): ───X^0.5───X^0.5───Y^0.5───Y^0.5───Y^0.5───,
 (8, 3): ───X^0.5───X^0.5───X^0.5───Y^0.5───,
 (8, 3): ───X^0.5───X^0.5───X^0.5───X^0.5───X^0.5───X^0.5───,
 (8, 3): ───X^0.5───X^0.5───X^0.5───Y^0.5───Y^0.5───Y^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───Y^0.5───X^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───Y^0.5───X^0.5───X^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───Y^0.5───X^0.5───X^0.5───X^0.5───,
 (8, 3): ───Y^0.5───Y^0.5───Y^0.5───Y^0.5───Y^0.5───Y^0.5